# Using NLTK to process reviews of beer to classify as "ipa-like", which is a proxy for hoppiness or "not ipa-like", which is a proxy for maltiness

In [6]:
## Python packages - you may have to pip install sqlalchemy, sqlalchemy_utils, and psycopg2.
%matplotlib inline
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
import psycopg2
import pandas as pd
import nltk
import numpy
import random

In [7]:
def main():
    ## Put stuff in notebook here
    print "main!"

In [8]:
#In Python: Define a database name (we're using a dataset on births, so I call it 
# birth_db), and your username for your computer (CHANGE IT BELOW). 
dbname = 'beer_db_2'
username = 'postgres'
mypassword = 'simple'
engine = create_engine('postgres://%s:%s@localhost/%s'%(username,mypassword,dbname))

con = psycopg2.connect(database = dbname, user = username,host='/var/run/postgresql',password=mypassword)

# Queries

In [9]:
beer_style_query = '''
SELECT
    breweries.style_name,count(distinct breweries.beer_name) as number_of_beers
FROM
    breweries
GROUP BY
    breweries.style_name
ORDER BY
    number_of_beers desc;
'''
beer_style_rank = pd.read_sql_query(beer_style_query,con)

beer_reviews_query = '''
SELECT
    reviews.review_text,
    reviews.beer_key,
    reviews.username,
    reviews.look,
    reviews.taste,
    reviews.feel,
    reviews.overall,
    breweries.style_key,
    breweries.style_name
FROM 
    reviews,breweries
WHERE
    breweries.beer_name_key = reviews.beer_key
ORDER BY
    reviews.username
'''
beer_reviews = pd.read_sql_query(beer_reviews_query,con)

ratings_per_user_query = '''
SELECT
    reviews.username,count(reviews.review_text) as number_of_reviews
FROM 
    reviews,breweries
WHERE
    reviews.beer_key = breweries.beer_name_key
GROUP BY
    reviews.username
ORDER BY 
    number_of_reviews desc;
'''

user_ratings = pd.read_sql_query(ratings_per_user_query,con)

# IPA, Double IPA, English IPA
ipa_query = """
SELECT
    reviews.username,
    reviews.review_text,
    reviews.stemmed_review_text,
    reviews.beer_key,
    breweries.brewery_key,
    breweries.beer_name,
    reviews.look,
    reviews.taste,
    reviews.feel,
    reviews.overall,
    breweries.style_key,
    breweries.style_name
FROM 
    reviews,breweries
WHERE
    breweries.beer_name_key = reviews.beer_key
AND breweries.ratings_count > 5
AND 
( 
 breweries.style_key = 116
 OR breweries.style_key = 140
 OR breweries.style_key = 150
)
ORDER BY
    breweries.avg_score desc;
"""

# american porter, english porter, oatmeal stout, milk/sweet stout
not_ipa_query = """
SELECT
    reviews.username,
    reviews.review_text,
    reviews.stemmed_review_text,
    reviews.beer_key,
    breweries.brewery_key,
    breweries.beer_name,
    reviews.look,
    reviews.taste,
    reviews.feel,
    reviews.overall,
    breweries.style_key,
    breweries.style_name
FROM 
    reviews,breweries
WHERE
    breweries.beer_name_key = reviews.beer_key
AND breweries.ratings_count > 5
AND 
(
 breweries.style_key = 159
 OR breweries.style_key = 101
 OR breweries.style_key = 67
 OR breweries.style_key = 82
)
ORDER BY
    reviews.overall desc;
"""

# All Beers Except Hoppy or Dark Superstyles
all_beers_query = """
SELECT
    reviews.username,
    reviews.review_text,
    reviews.stemmed_review_text,
    reviews.beer_key,
    breweries.brewery_key,
    breweries.beer_name,
    reviews.look,
    reviews.taste,
    reviews.feel,
    reviews.overall,
    breweries.style_key,
    breweries.style_name
FROM 
    reviews,breweries
WHERE
    breweries.beer_name_key = reviews.beer_key
AND breweries.ratings_count > 0
ORDER BY
    reviews.overall desc;
"""
untrained_beers_query = """
SELECT
    reviews.username,
    reviews.review_text,
    reviews.stemmed_review_text,
    reviews.beer_key,
    breweries.brewery_key,
    breweries.beer_name,
    reviews.look,
    reviews.taste,
    reviews.feel,
    reviews.overall,
    breweries.style_key,
    breweries.style_name
FROM 
    reviews,breweries
WHERE
    breweries.beer_name_key = reviews.beer_key
AND breweries.ratings_count > 0
AND
(
 breweries.style_key != 159
 AND breweries.style_key != 101
 AND breweries.style_key != 67
 AND breweries.style_key != 82
 AND breweries.style_key != 116
 AND breweries.style_key != 140
 AND breweries.style_key != 150
)
ORDER BY
    reviews.overall desc;
"""

# Extracting Data From the Queries

In [10]:
ipa = pd.read_sql_query(ipa_query,con)
not_ipa = pd.read_sql_query(not_ipa_query,con)
all_beers = pd.read_sql_query(all_beers_query,con)

In [11]:
print 'ipa reviews:',ipa.shape[0]
print 'not_ipa reviews:',not_ipa.shape[0]
print 'all reviews:',all_beers.shape[0]

ipa reviews: 7557
not_ipa reviews: 1471
all reviews: 33827


In [14]:
# Put this stuff into a form that nltk can use
beer_reviews = []
hoppy_reviews = []
malty_reviews = []
all_beer_reviews = []

all_words = []
hoppy_words = []
malty_words = []
counter = 0

# choose whether to use stemmed_review_text or review_text
#which_review = 'stemmed_review_text'
which_review = 'review_text'

for index,row in all_beers.iterrows():
    #print row[which_review]
    try:
        review_words = row[which_review].split()
        all_beer_reviews.append(review_words)
    except:
        print "skipping, no review data"
print "extracted",len(all_beer_reviews)

for index,row in ipa.iterrows():
    try:
        review_words = row[which_review].split()
        beer_reviews.append((review_words,"ipa"))
        hoppy_reviews.append((review_words,"ipa"))
        for w in review_words:
            all_words.append(w.lower())
            hoppy_words.append(w.lower())
        counter += 1
        if counter == 500:
            print counter,"hoppy reviews gotten"
            break
    except:
        "skipping, no review data" 

counter = 0

for index,row in not_ipa.iterrows():
    try:
        review_words = row[which_review].split()
        beer_reviews.append((review_words,"not_ipa"))
        malty_reviews.append((review_words,"not_ipa"))
        for w in review_words:
            all_words.append(w.lower())
            malty_words.append(w.lower())
        counter += 1
        if counter == 500:
            print counter,"malty top reviews gotten"
            break
    except:
        "skipping, no review data"
print "got",len(beer_reviews),'reviews, with',len(all_words),'words.'

skipping, no review data
extracted 33826
500 hoppy reviews gotten
500 malty top reviews gotten
got 1000 reviews, with 63755 words.


In [15]:
random.shuffle(beer_reviews)

# Compile Words

Here, we can get the frequency distribution of keywords, which are not stemmed.

In [16]:
all_words = nltk.FreqDist(all_words)
hoppy_words = nltk.FreqDist(hoppy_words)
malty_words = nltk.FreqDist(malty_words)
print "Hoppy Words:",hoppy_words.most_common(30),"\n"
print "Malty Words:",malty_words.most_common(30),"\n"
print "All words:",all_words.most_common(30),"\n"

Hoppy Words: [('i', 777), ('hops', 312), ('the', 311), ('citrus', 302), ('taste', 274), ('ipa', 267), ('hop', 263), ('head', 256), ('orange', 250), ('tropical', 237), ('good', 214), ('grapefruit', 206), ('a', 198), ('like', 198), ('fruit', 192), ('pine', 191), ('malt', 189), ('nice', 183), ('white', 182), ('one', 176), ('this', 171), ('light', 169), ('finish', 168), ('flavor', 163), ('great', 161), ('aroma', 160), ('bitterness', 156), ('carbonation', 154), ('well', 154), ('medium', 153)] 

Malty Words: [('i', 664), ('coffee', 647), ('chocolate', 515), ('dark', 470), ('the', 414), ('head', 370), ('taste', 299), ('porter', 296), ('black', 277), ('like', 275), ('roasted', 272), ('nice', 272), ('one', 227), ('a', 226), ('flavor', 224), ('vanilla', 223), ('malt', 218), ('good', 215), ('brown', 212), ('light', 212), ('well', 198), ('this', 183), ('sweet', 182), ('smooth', 179), ('creamy', 168), ('great', 166), ('medium', 165), ('bit', 161), ('finish', 157), ('aroma', 154)] 

All words: [('i'

# Create sparse vectors of words for bayesian classificaiton

In [17]:
word_features = list(all_words.keys())[:3000]

def find_features(beer_review):
    words = set(beer_review)
    features = {}
    for w in word_features:
        features[w] = (w in words) # boolean assignment
    return features
print find_features(malty_reviews[0][0])

{'clamoring': False, 'desirable': False, 'yellow': False, 'four': False, 'maltier': False, 'woods': False, 'hanging': False, 'ringlets': False, 'woody': False, 'centimeter': False, 'worked': False, 'adorning': False, 'payoff': False, 'increase': False, 'quaffable': False, 'canes': False, 'buddy': False, 'quadruple': False, 'tickle': False, 'tingle': False, 'swap': False, 'lord': False, 'gracious': False, 'worth': False, 'deli': False, 'blanket': False, 'manic': False, 'expierences': False, 'figs': False, 'bringing': False, 'caramels': False, 'basics': False, 'daydream': False, 'caramely': False, 'pinch': False, 'specialties': False, 'differentiates': False, 'nondescript': False, 'shows': False, 'clothes': False, 'chew': False, 'specially': False, 'tired': False, 'blasphemy': False, 'glasss': False, 'preface': False, 'bacon': False, 'elegant': False, 'second': False, 'glassl': False, 'rigorous': False, 'glassa': False, 'beigebrown': False, 'lucious': False, 'filtration': False, 'pace': 

In [21]:
featuresets = [(find_features(rev),category) for (rev,category) in beer_reviews]
training_set = featuresets[:500]
testing_set = featuresets[500:]
print len(featuresets)

1000


# Bayes Classifier

## Train the classifier

In [19]:
classifier = nltk.NaiveBayesClassifier.train(training_set)

## Run the classifier on the testing set, determine accuracy

In [20]:
print "Naive Bayes Algo % accuracy:",(nltk.classify.accuracy(classifier,testing_set))*100.
classifier.show_most_informative_features(30)

Naive Bayes Algo % accuracy: 94.6
Most Informative Features
                  coffee = True           not_ip : ipa    =     60.2 : 1.0
                 roasted = True           not_ip : ipa    =     52.2 : 1.0
              grapefruit = True              ipa : not_ip =     49.4 : 1.0
                  roasty = True           not_ip : ipa    =     21.6 : 1.0
                   piney = True              ipa : not_ip =     16.7 : 1.0
                    dark = True           not_ip : ipa    =     15.3 : 1.0
                  fruity = True              ipa : not_ip =     14.6 : 1.0
                     ipa = True              ipa : not_ip =     13.2 : 1.0
                  barrel = True           not_ip : ipa    =     12.8 : 1.0
                   mocha = True           not_ip : ipa    =     10.3 : 1.0
                   edges = True           not_ip : ipa    =      9.1 : 1.0
                    hype = True              ipa : not_ip =      8.9 : 1.0
                    gold = True         

## Now, we can classify individual beers using the bayesean classifier

In [ ]:
def get_hoppy_probabilities(feature_set):
    prob = {}
    dist = classifier.prob_classify(feature_set)
    for label in dist.samples():
        prob[label] = dist.prob(label)
    return prob

In [ ]:
print classifier.prob_classify(find_features(malty_reviews[0][0]))
print classifier.classify(find_features(malty_reviews[0][0]))

#probs = get_hoppy_probabilities(test_features)
#print probs["ipa"],probs["not_ipa"]

# for review in malty_reviews:
#     dist = classifier.prob_classify(find_features(review[0]))
#     for label in dist.samples():
#         print("%s: %f" % (label, dist.prob(label)))

In [ ]:
rows_list = []
index_out = 0
for index,row in all_beers.iterrows():
    #print index
    #print row
    try:
        features = find_features(row['review_text'].split())
    except:
        continue
    hoppiness = get_hoppy_probabilities(features)
    #print hoppiness
    
    out_dict = {}
    out_dict['beer_key'] = row['beer_key']
    out_dict['username'] = row['username']
    out_dict['review_text'] = row['review_text']
    out_dict['name'] = row['beer_name']
    out_dict['style'] = row['style_name']
    out_dict['hoppiness'] = hoppiness['ipa']
    out_dict['maltiness'] = hoppiness['not_ipa']
    out_dict['hop_liklihood'] = hoppiness['ipa']/(hoppiness['ipa']+hoppiness['not_ipa'])
    rows_list.append(out_dict)
    #print rows_list

In [ ]:
classified_df = pd.DataFrame(rows_list)

In [ ]:
classified_df['hoppiness'].plot(kind='hist',figsize=(8,8))

In [ ]:
classified_df['hoppiness'][classified_df['style']=='Kölsch'].plot(kind='hist',figsize=(8,8))

In [ ]:
classified_df['hoppiness'][classified_df['style']=='Saison / Farmhouse Ale'].plot(kind='hist',figsize=(8,8))

In [ ]:
classified_df['hoppiness'][classified_df['style']=='American IPA'].plot(kind='hist',figsize=(8,8))

In [ ]:
classified_df['hoppiness'][classified_df['style']=='American Porter'].plot(kind='hist',figsize=(8,8))

In [ ]:
#classified_df.sort(['maltiness'],ascending=0)

In [ ]:
classified_beers = []
classified_df.to_csv("test.csv")
